In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# Prevent wrapping on tables for readability
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

sk.set_file_destination("trend_example")
sk.login("SALIENT_USERNAME", "SALIENT_PASSWORD")

## Customize the Calculation

Set variables that will influence how the notebook behaves.


In [ ]:
# The variables to project into the future:
var = ["tmax", "tmin", "precip"]

# When to start the future timeseries.  Valid values:
# * A pd.Timestamp
# * A string in YYYY-MM-DD format
# * -today (the default)
start_date = "-today"

# When to end the future timeseries.  Valid values:
# * a date
# * a pd.Timestamp
# * a DateOffset relative to start_date
end_date = pd.DateOffset(years=5)

# Set force to "True" to disable caching
force = False

The later call to `data_timeseries` will accept a `sk.Location` that is a single point, a vector of multiple points (shown here), or a `shapefile` polygon.


#### Define the area of interest

For this analysis, we'll generate a trend for a selection of global airports.


In [ ]:
geo = pd.DataFrame(
    [
        # Europe
        (2.359, 48.725, "CDG", "Charles de Gaulle", "France"),
        (-0.462, 51.470, "LHR", "Heathrow", "United Kingdom"),
        (23.944, 37.936, "ATH", "Athens Intl", "Greece"),
        (12.250, 41.804, "FCO", "Leonardo da Vinci", "Italy"),
        (37.416, 55.972, "SVO", "Sheremetyevo", "Russia"),
        (24.963, 60.317, "HEL", "Helsinki-Vantaa", "Finland"),
        (8.550, 50.033, "FRA", "Frankfurt", "Germany"),
        (4.764, 52.308, "AMS", "Schiphol", "Netherlands"),
        (13.292, 52.559, "TXL", "Tegel", "Germany"),
        (19.258, 47.439, "BUD", "Ferenc Liszt", "Hungary"),
        # Africa
        (31.406, 30.111, "CAI", "Cairo Intl", "Egypt"),
        (7.266, 9.007, "ABV", "Nnamdi Azikiwe", "Nigeria"),
        (30.130, -1.968, "KGL", "Kigali Intl", "Rwanda"),
        (28.246, -26.139, "JNB", "O.R. Tambo", "South Africa"),
        (3.361, 6.577, "LOS", "Murtala Muhammed", "Nigeria"),
        (-7.590, 33.367, "CMN", "Mohammed V", "Morocco"),
        (39.252, -6.878, "DAR", "Julius Nyerere", "Tanzania"),
        (36.928, -1.319, "NBO", "Jomo Kenyatta", "Kenya"),
        # Middle East
        (55.364, 25.252, "DXB", "Dubai Intl", "United Arab Emirates"),
        (51.565, 25.273, "DOH", "Hamad Intl", "Qatar"),
        (46.698, 24.966, "RUH", "King Khalid", "Saudi Arabia"),
        (35.389, 31.722, "AMM", "Queen Alia", "Jordan"),
        # Asia
        (103.989, 1.359, "SIN", "Singapore Changi", "Singapore"),
        (101.709, 2.745, "KUL", "Kuala Lumpur", "Malaysia"),
        (113.915, 22.309, "HKG", "Hong Kong Intl", "China"),
        (140.386, 35.772, "NRT", "Narita", "Japan"),
        (121.233, 31.197, "PVG", "Pudong", "China"),
        (126.451, 37.469, "ICN", "Incheon", "South Korea"),
        (116.587, 40.080, "PEK", "Beijing Capital", "China"),
        (72.868, 19.089, "BOM", "Chhatrapati Shivaji", "India"),
        (77.103, 28.556, "DEL", "Indira Gandhi", "India"),
        (100.747, 13.681, "BKK", "Suvarnabhumi", "Thailand"),
        (106.652, 10.818, "SGN", "Tan Son Nhat", "Vietnam"),
        (96.133, 16.907, "RGN", "Yangon Intl", "Myanmar"),
        # Oceania
        (151.177, -33.946, "SYD", "Kingsford Smith", "Australia"),
        (144.843, -37.673, "MEL", "Melbourne", "Australia"),
        (115.967, -31.940, "PER", "Perth", "Australia"),
        (174.785, -37.008, "AKL", "Auckland", "New Zealand"),
        (147.220, -9.443, "POM", "Port Moresby", "Papua New Guinea"),
        (-157.922, 21.332, "HNL", "Daniel K. Inouye", "United States"),
        # South America
        (-46.472, -23.432, "GRU", "Guarulhos", "Brazil"),
        (-58.535, -34.822, "EZE", "Ministro Pistarini", "Argentina"),
        (-70.786, -33.393, "SCL", "Arturo Merino", "Chile"),
        (-74.147, 4.701, "BOG", "El Dorado", "Colombia"),
        (-79.619, -2.157, "GYE", "José Joaquín", "Ecuador"),
        (-66.991, 10.601, "CCS", "Simón Bolívar", "Venezuela"),
        (-77.114, -12.024, "LIM", "Jorge Chávez", "Peru"),
        # Central America & Caribbean
        (-99.072, 19.436, "MEX", "Benito Juárez", "Mexico"),
        (-79.383, 8.979, "PTY", "Tocumen", "Panama"),
        (-82.409, 23.034, "HAV", "José Martí", "Cuba"),
        (-69.669, 18.430, "SDQ", "Las Américas", "Dominican Republic"),
        # North America
        (-73.778, 45.471, "YUL", "Montréal-Trudeau", "Canada"),
        (-79.612, 43.677, "YYZ", "Toronto Pearson", "Canada"),
        (-114.020, 51.114, "YYC", "Calgary", "Canada"),
        (-123.184, 49.194, "YVR", "Vancouver", "Canada"),
        (-122.375, 37.619, "SFO", "San Francisco", "United States"),
        (-87.905, 41.979, "ORD", "O'Hare", "United States"),
        (-80.290, 25.795, "MIA", "Miami", "United States"),
        (-73.779, 40.641, "JFK", "John F Kennedy", "United States"),
        (-118.408, 33.942, "LAX", "Los Angeles", "United States"),
        (-95.341, 29.984, "IAH", "George Bush", "United States"),
        (-84.428, 33.640, "ATL", "Hartsfield-Jackson", "United States"),
    ],
    columns=["lon", "lat", "iata", "full_name", "country"],
)

### Define the Location


In [ ]:
loc = sk.Location(
    location_file=sk.upload_location_file(
        lats=geo.lat,
        lons=geo.lon,
        names=geo.iata,
        country=geo.country,
        full_name=geo.full_name,
        geoname="airports_global",
        force=force,
    )
)
print(loc.load_location_file().drop("geometry", axis=1))
# loc.plot_locations(names=False, title="World Airports")

       lat      lon name         country           full_name
0   48.725    2.359  CDG          France   Charles de Gaulle
1   51.470   -0.462  LHR  United Kingdom            Heathrow
2   37.936   23.944  ATH          Greece         Athens Intl
3   41.804   12.250  FCO           Italy   Leonardo da Vinci
4   55.972   37.416  SVO          Russia        Sheremetyevo
..     ...      ...  ...             ...                 ...
57  25.795  -80.290  MIA   United States               Miami
58  40.641  -73.779  JFK   United States      John F Kennedy
59  33.942 -118.408  LAX   United States         Los Angeles
60  29.984  -95.341  IAH   United States         George Bush
61  33.640  -84.428  ATL   United States  Hartsfield-Jackson

[62 rows x 5 columns]


## Extrapolate the Climatological Trend

The function `extrapolate_trend` will use Salient's 30-year linear trend to generate a per-day timeseries on for any date range.


In [ ]:
trend = sk.data_timeseries_api.extrapolate_trend(
    loc=loc,
    variable=var,
    start=start_date,
    end=end_date,
    force=force,
)

print(trend)

<xarray.Dataset> Size: 3MB
Dimensions:   (location: 62, time: 1827)
Coordinates:
  * location  (location) object 496B 'CDG' 'LHR' 'ATH' ... 'LAX' 'IAH' 'ATL'
    lat       (location) float64 496B 48.73 51.47 37.94 ... 33.94 29.98 33.64
    lon       (location) float64 496B 2.359 -0.462 23.94 ... -95.34 -84.43
  * time      (time) datetime64[ns] 15kB 2025-04-04 2025-04-05 ... 2030-04-04
Data variables:
    precip    (time, location) float64 906kB 1.265 1.367 1.013 ... 1.922 3.903
    tmax      (time, location) float64 906kB 15.68 13.12 19.83 ... 26.56 23.0
    tmin      (time, location) float64 906kB 5.675 5.148 10.84 ... 15.91 11.29


### Visualize the Results

View the trend as a set of stacked anomalies.


In [ ]:
ds_plot = (
    trend.assign_coords(year=trend.time.dt.year, dayofyear=trend.time.dt.dayofyear)
    .set_index({"time": ["year", "dayofyear"]})
    .unstack("time")
    .mean("location", keep_attrs=True)
)

# Drop first (incomplete) year and calculate anomalies relative to first complete year
first_year = ds_plot.year.min().item()
ds_plot = ds_plot.sel(year=slice(first_year + 1, None))
with xr.set_options(keep_attrs=True):
    ds_plot = ds_plot - ds_plot.sel(year=first_year + 1)

fig, axes = plt.subplots(len(ds_plot.data_vars), 1, figsize=(12, 6 * len(ds_plot.data_vars)))
for (var_name, da), ax in zip(ds_plot.data_vars.items(), axes):
    da.plot(x="dayofyear", hue="year", ax=ax)
    ax.set_title(f"Average {var_name.capitalize()} Change from {first_year + 1}")

### Export in Tabular Form

If you wish to upload the results to a database, a tabular representation may be more natural.


In [ ]:
trend_table = trend.to_dataframe().reorder_levels(["time", "location"]).sort_index()
file_table = os.path.join(sk.get_file_destination(), "trend_table.csv")
trend_table.to_csv(file_table)

print(f"Wrote table to '{file_table}':")
print(trend_table)

Wrote table to 'trend_example/trend_table.csv':
                        lat      lon    precip       tmax       tmin
time       location                                                 
2025-04-04 ABV        9.007    7.266  0.990180  35.825071  26.173296
           AKL      -37.008  174.785  2.692716  21.465929  15.890984
           AMM       31.722   35.389  0.593724  24.602078  14.035481
           AMS       52.308    4.764  1.273668  12.219561   5.446900
           ATH       37.936   23.944  1.012770  19.831917  10.835725
...                     ...      ...       ...        ...        ...
2030-04-04 TXL       52.559   13.292  0.922891  13.920495   4.369089
           YUL       45.471  -73.778  3.630097   7.354522  -1.126916
           YVR       49.194 -123.184  7.382996  10.169097   5.920988
           YYC       51.114 -114.020  1.371740   7.537110  -3.919986
           YYZ       43.677  -79.612  3.598309   8.290379   0.306223

[113274 rows x 5 columns]
